In [124]:
import pickle
import nltk
import math
import os

In [125]:
class vect:
    SCALAR = 1
    COSINE = 2
    JACCARD = 3

class Docs:

    def __init__(self,Lancaster=True,split=False):

        docs = []
        self.maxfreq = {}
        MotsVides = nltk.corpus.stopwords.words('english')
        self.Lancaster = Lancaster
        self.split = split

        files = os.listdir("./TP2data/")
        self.txt_files = [file for file in files if file.endswith(".txt")]
        file = ""
        my_dict = {}
        for i in range(len(self.txt_files)):
            
            file = open("./TP2data/"+self.txt_files[i]).read()
            file = file.replace("\n","")
            Termes = self.splitter(file)

            TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
            TermesNormalisation = [self.normalize(terme) for terme in TermesSansMotsVides]

            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict


        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if keys[1] not in self.maxfreq.keys():
                self.maxfreq[keys[1]] = 0
            if self.dict[keys] > self.maxfreq[keys[1]]:
                self.maxfreq[keys[1]] = self.dict[keys]

        self.N = len(docs)
    
    def rebuild(self,Lancaster=True,split=False):
        docs = []
        self.maxfreq = {}
        MotsVides = nltk.corpus.stopwords.words('english')
        self.Lancaster = Lancaster
        self.split = split
        
        file = ""
        my_dict = {}
        for i in range(len(self.txt_files)):
            file = open("./TP2data/"+self.txt_files[i]).read()
            file = file.replace("\n","")

            Termes = self.splitter(file)

            TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
            TermesNormalisation = [self.normalize(terme) for terme in TermesSansMotsVides]

            for term in TermesNormalisation:
                
                if (term.lower(),i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict


        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if keys[1] not in self.maxfreq.keys():
                self.maxfreq[keys[1]] = 0
            if self.dict[keys] > self.maxfreq[keys[1]]:

                self.maxfreq[keys[1]] = self.dict[keys]

        self.N = len(docs)
        self.DictWeight()
    def normalize(self,text):
        if self.Lancaster :
            Lancaster = nltk.LancasterStemmer()
            text = Lancaster.stem(text)
        else :
            porter = nltk.PorterStemmer()
            text = porter.stem(text)
        return text
    def splitter(self,text):
        if not self.split:
            return text.split(" ")
        else:
            ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
            Termes = ExpReg.tokenize(text) 
            return Termes
    def calcWeight(self,word,doc):

        inDocs = []

        for keys in self.dict.keys():
            if keys[0] == word:
                if keys[1] not in inDocs:
                    inDocs.append(keys[1])

        # nombre de documents contenant le mot
        Ni = len(inDocs)

        try:
            tf = self.dict[(word,doc)]/self.maxfreq[doc]
        except:
            tf = 0
            
        #idf = math.log10((N/Ni)+1)
        try:
            idf = math.log10((self.N/Ni)+1)
        except:
            idf = 0

        return(tf*idf)
    def DictWeight(self):

        self.TDFP = [(word,doc,self.dict[(word,doc)],self.calcWeight(word,doc)) for word,doc in self.dict.keys()]
        self.DTFP = [(doc,word,freq,weight) for word,doc,freq,weight in self.TDFP]
        self.TDFP = sorted(self.TDFP, key=lambda x: x[0])
    def MV(self,termes,typeV = vect.SCALAR):
        rsv = {}
        termes = self.splitter(termes)
        normterms = []
        for term in termes:
            normterms.append(self.normalize(term))

        for doc in range(self.N):
            QD = 0
            v2 = 0
            w2 = 0

            for term in normterms:
                if (term.lower(),doc) in self.dict.keys():
                    weight = self.calcWeight(term,doc)
                    QD+= weight
                    w2+= weight**2
                    v2+= 1**2
            try:
                if typeV == vect.COSINE:
                    QD = QD / (math.sqrt(v2)*math.sqrt(w2))
                if typeV == vect.JACCARD:
                    QD = QD / (w2+v2 - QD)
            except ZeroDivisionError:
                QD = 0
            rsv[doc] = QD

        rsv = dict(sorted(rsv.items(), key=lambda item: item[1],reverse=True))
        return rsv

    def bm25(self,termes,K,B):
        rsv = {}
        #tokenization de la requete
        termes = self.splitter(termes)
        normterms = []
        dls = {}
        #normalisation de la requete 
        for term in termes:
            normterms.append(self.normalize(term.lower()))

        # calcule nombre termes par doc
        for term,doc in self.dict.keys():
            if doc in dls.keys():
                dls[doc] +=1
            else:
                dls[doc] = 1

        # moyenne des termes par doc
        avdl = 0
        for key in dls.keys():
            avdl += dls[key]
        avdl = avdl / len(dls.keys())

        for doc in range(self.N):
            QD = 0
            for term in normterms:
                val = 0
                log = 0
                if (term,doc) in self.dict.keys():
                    ni = 0
                    
                    for doc2 in range(self.N):
                        if (term.lower(),doc2) in self.dict.keys():
                            ni+=1
                    freq = self.dict[(term.lower(),doc)]
                    log = math.log10((self.N - ni + 0.5) / (ni + 0.5))
                    val = freq + K*((1-B)+(B*dls[doc]/avdl))
                    val = freq/val
                QD+= val*log
            rsv[doc] = QD
            
        #trie du dictionnaire
        rsv = dict(sorted(rsv.items(), key=lambda item: item[1],reverse=True))
        return rsv
    def boolean(self,text):
        text = self.splitter(text)
        docText = {}
        for doc in range(d.N):
            copy = text.copy()
            lexique = [" * "," - "," + "," 1 "]
            copy = [" * " if item == "AND" else item for item in copy]
            copy = [" - " if item == "NOT" else item for item in copy]
            copy = [" + " if item == "OR" else item for item in copy]
            copy = [" 1 " if (self.normalize(item),doc) in self.dict.keys() else item for item in copy]
            copy = [" 0 " if item not in lexique  else item for item in copy]
            new_copy = []
            i=0
            while(i<len(copy)):
                if copy[i] == " - " and copy[i+1] == " 0 ":
                    new_copy.append(" 1 ")
                    i+=2
                elif copy[i] == " - " and copy[i+1] == " 1 ":
                    new_copy.append(" 0 ")
                    i+=2
                else:
                    new_copy.append(copy[i])
                    i+=1
            docText[doc] = 1 if eval("".join(new_copy)) > 0 else 0
        return docText
d = Docs(Lancaster=False,split=True)
d.DictWeight()

d.TDFP



<>:100: SyntaxWarning: invalid escape sequence '\.'
<>:100: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Mekki\AppData\Local\Temp\ipykernel_12976\2298217475.py:100: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>>


[('0.6%', 4, 1, 0.16901960800285137),
 ('04', 5, 1, 0.07682709454675062),
 ('100', 2, 2, 0.2112745100035642),
 ('17', 2, 1, 0.1056372550017821),
 ('2019', 0, 1, 0.16901960800285137),
 ('2020', 0, 1, 0.16901960800285137),
 ('4,000', 2, 1, 0.1056372550017821),
 ('4.2%', 4, 1, 0.16901960800285137),
 ('500x', 2, 1, 0.1056372550017821),
 ('7b', 0, 1, 0.16901960800285137),
 ('abil', 2, 1, 0.1056372550017821),
 ('achiev', 0, 1, 0.16901960800285137),
 ('addit', 1, 1, 0.059640156839957804),
 ('addit', 2, 1, 0.059640156839957804),
 ('addit', 5, 1, 0.043374659519969314),
 ('address', 0, 1, 0.07958800173440753),
 ('address', 1, 1, 0.0497425010840047),
 ('address', 2, 1, 0.0497425010840047),
 ('address', 5, 1, 0.03617636442473069),
 ('adopt', 5, 1, 0.07682709454675062),
 ('advent', 1, 1, 0.1056372550017821),
 ('aim', 5, 1, 0.07682709454675062),
 ('allow', 3, 1, 0.06689555459199582),
 ('allow', 5, 1, 0.05473272648436022),
 ('also', 1, 1, 0.0497425010840047),
 ('also', 2, 1, 0.0497425010840047),
 ('a

In [126]:
import tkinter as tk
from tkinter import ttk

def search():
    global stemming

    text = query_text.get()
    table.delete(*table.get_children())
    if index_var.get() == 0:
        data = d.TDFP
    else:
        data = d.DTFP

    if text == "":
        print("empty")
        for terme in data:
            if type(terme[0]) is str:
                if text.lower() in terme[0].lower():
                    table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
            else:
                if text.lower() in terme[1].lower():
                    table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
        return
    
    searchText = d.splitter(text)
    searchText = [d.normalize(terme) for terme in searchText]

    for text in searchText:
        for terme in data:
            if type(terme[0]) is str:
                if text.lower() in terme[0].lower():
                    table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])
            else:
                if text.lower() in terme[1].lower():
                    table.insert("", "end", values=[terme[0],terme[1],terme[2],terme[3]])

def on_token_change(*args):
    d.rebuild(Lancaster=stemming.get(),split=token.get())
    search()

def on_index_change(*args):
    search()
def show_match():
    text = query_text.get()
    table.delete(*table.get_children())

    if selected_option.get() == "SCALAR":
        rsv = d.MV(text,vect.SCALAR)
    if selected_option.get() == "COSINE":
        rsv = d.MV(text,vect.COSINE)
    if selected_option.get() == "JACCARD":
        rsv = d.MV(text,vect.JACCARD)

    for doc in rsv.keys():
        table.insert("", "end", values=[doc,"",rsv[doc],""])
def show_bm25():
    text = query_text.get()
    table.delete(*table.get_children())
    b = float(input_B.get())
    k = float(input_K.get())
    rsv = d.bm25(text,k,b)
    for doc in rsv.keys():
        table.insert("", "end", values=[doc,"",rsv[doc],""])
def show_boolean_model():
    global stemming
    table.delete(*table.get_children())
    text = query_text.get()
    docText = d.boolean(text)
    for doc in docText.keys():
        table.insert("", "end", values=[doc,"",docText[doc],""])
def show_datamining_model():
    
    pass
# Create the main application window
root = tk.Tk()
root.title("Text Processing Application")

# Search Query Text Box and Search Button
search_label_frame = ttk.LabelFrame(root, text="Search")
search_label_frame.grid(row=0, column=0,columnspan=2, padx=10, pady=10)
query_label = tk.Label(search_label_frame, text="Search Query:")
query_label.grid(row=0, column=0)
query_text = tk.Entry(search_label_frame, width=40)
query_text.grid(row=0, column=1)
search_button = tk.Button(search_label_frame, text="Search", command=search)
search_button.grid(row=0, column=2)

# Border around "Processing" section
processing_label_frame = ttk.LabelFrame(root, text="Processing")
processing_label_frame.grid(row=1, column=1, padx=10, pady=10)

# Radio buttons for Processing
stemming = tk.IntVar()
stemming.trace("w", on_token_change)

token = tk.IntVar()
token.trace("w", on_token_change)

tokenization_radiobutton = tk.Checkbutton(processing_label_frame, text="Tokenization", variable=token)
porter_stemmer_radiobutton = tk.Checkbutton(processing_label_frame, text="Lancaster Stemmer", variable=stemming)
porter_stemmer_radiobutton.select()
tokenization_radiobutton.select()
tokenization_radiobutton.grid(row=0, column=0)
porter_stemmer_radiobutton.grid(row=1, column=0)

# Border around "Index" section
index_label_frame = ttk.LabelFrame(root, text="Index")
index_label_frame.grid(row=1, column=0, padx=10, pady=10)
index_var = tk.IntVar()
index_var.trace("w", on_index_change)
docs_per_term_radiobutton = tk.Radiobutton(index_label_frame, text="DOCS per TERM", variable=index_var, value=0)
terms_per_doc_radiobutton = tk.Radiobutton(index_label_frame, text="TERMS per DOC", variable=index_var, value=1)
docs_per_term_radiobutton.grid(row=1, column=0)
terms_per_doc_radiobutton.grid(row=2, column=0)

# Table to display data
table_frame = ttk.Frame(root)
table_frame.grid(row=2, column=0, columnspan=2,rowspan=2, pady=10)
table = ttk.Treeview(table_frame, columns=("1", "2", "3", "4"))
table.heading("#1", text="terme")
table.heading("#2", text="document")
table.heading("#3", text="freq")
table.heading("#4", text="Score")
table['show'] = 'headings'
# Vertical scrollbar for the table
table_scrollbar = ttk.Scrollbar(table_frame, orient="vertical", command=table.yview)
table.configure(yscrollcommand=table_scrollbar.set)

# Grid layout for the table and scrollbar
table.grid(row=0, column=0, sticky="nsew")
table_scrollbar.grid(row=0, column=1, sticky="ns")


# Make the table resizable
table_frame.grid_rowconfigure(0, weight=1)
table_frame.grid_columnconfigure(0, weight=1)

# ----------------------------------
# Border around "Processing" section
# ----------------------------------
Matching_label_frame = ttk.LabelFrame(root, text="Matching")
Matching_label_frame.grid(row=1, column=2,rowspan=2, padx=10, pady=10)
# Vector space model
VectorSpaceModel_label_frame = ttk.LabelFrame(Matching_label_frame, text="Vector space model")
VectorSpaceModel_label_frame.grid(row=0, column=0, padx=10, pady=10)
selected_option = tk.StringVar()
combo = ttk.Combobox(VectorSpaceModel_label_frame, textvariable=selected_option, state='readonly')
combo['values'] = ('SCALAR','JACCARD', 'COSINE')  
combo.set('SCALAR') 
combo.grid(row=0, column=0)
show_matching_button = tk.Button(VectorSpaceModel_label_frame, text="Vector space model", command=show_match)
show_matching_button.grid(row=1, column=0)
# BM 25
bm25_label_frame = ttk.LabelFrame(Matching_label_frame, text="Bm25")
bm25_label_frame.grid(row=0, column=1, padx=10, pady=10)
tk.Label(bm25_label_frame, text="K").grid(row=0, column=0)
input_K = tk.Entry(bm25_label_frame,width=20)
input_K.grid(row=0, column=1)
tk.Label(bm25_label_frame, text="B").grid(row=1, column=0)
input_B = tk.Entry(bm25_label_frame, width=20)
input_B.grid(row=1, column=1)
show_bm25_button = tk.Button(bm25_label_frame, text="Bm25", command=show_bm25)
show_bm25_button.grid(row=2, column=1)
# boolean model
boolean_label_frame = ttk.LabelFrame(Matching_label_frame, text="Boolean model")
boolean_label_frame.grid(row=1, column=0, padx=10, pady=10)
boolean_model_button = tk.Button(boolean_label_frame, text="Boolean model", command=show_boolean_model)
boolean_model_button.grid(row=0, column=0)

# boolean model
Datamining_label_frame = ttk.LabelFrame(Matching_label_frame, text="Datamining model")
Datamining_label_frame.grid(row=1, column=1, padx=10, pady=10)
Datamining_model_button = tk.Button(Datamining_label_frame, text="Datamining model", command=show_datamining_model)
Datamining_model_button.grid(row=0, column=0)

for dat in d.TDFP:
    table.insert("", "end", values=[dat[0],dat[1],dat[2],dat[3]])

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\APPS\python\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_12976\2208073675.py", line 40, in on_token_change
    search()
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_12976\2208073675.py", line 8, in search
    table.delete(*table.get_children())
                  ^^^^^^^^^^^^^^^^^^^^
  File "d:\APPS\python\Lib\tkinter\ttk.py", line 1195, in get_children
    self.tk.call(self._w, "children", item or '') or ())
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: invalid command name ".!frame.!treeview"
Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\APPS\python\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Mekki\AppData\Local\Temp\ipykernel_12976\2208073675.py", line 40, in on_t

empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
